# Taller Python
## Python for Exploratory Data Analysis

## 1. Modules

In [ ]:
# requirements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno
import geopandas as gpd
import geoplot as gplt
from geopy import distance
import mapclassify as mc
# common
import os
from pathlib import Path
from ast import literal_eval
# user defined
from modules.utils import percentPlot
from modules.utils import km_bpoints

# settings
colors = [
    '#00AF66',
    '#F0BE32',
    '#DB4437',
    '#CCCCCC',
    '#4285F4',
    '#8E6495',
    '#FF7C55'
]

os.chdir(r'../')
Path('temp').mkdir(parents=True, exist_ok=True)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

sns.set_theme()
sns.set_palette(sns.color_palette(colors))

## 2. Data inspection

### 2.1. Pandas

#### 2.1.1. pandas.DataFrame.info

+ *Información básica sobre dataframe*

In [ ]:
df_casas = pd.read_csv('./data/casas_data.csv', delimiter=';')
df_casas.info()

#### 2.1.2. pandas.DataFrame.head

+ *Primeras filas del dataframe*

In [ ]:
df_casas.head(5)

### 2.2. Geopandas

+ *Manipulación de datos georrefenciados*

#### 2.2.1. geopandas.GeoDataFrame

In [ ]:
geoData = gpd.GeoDataFrame(
    df_casas.copy(),
    geometry=gpd.points_from_xy(df_casas['longitud'], df_casas['latitud'])
)
geoData.head(3)

#### 2.2.2. geopandas.GeoDataFrame.plot

+ *Integración con shapefiles*

In [ ]:
shp_peru = gpd.read_file('./data/shapefiles/DISTRITOS.shp')
shp_peru['IDDIST'] = shp_peru['IDDIST'].astype('int')

shp_peru.plot(figsize=(15,15))

### 2.3. Main dataset

#### 2.3.1. Filtros

+ *Solo Lima Metropolitana y Callao (50 distritos)*

In [ ]:
lima_shp_filter = (shp_peru['DEPARTAMEN'] == 'LIMA') & (shp_peru['PROVINCIA'] == 'LIMA')
callao_shp_filter = (shp_peru['DEPARTAMEN'] == 'CALLAO')

shp_lima = shp_peru[lima_shp_filter | callao_shp_filter]

print('Number distritos:', len(shp_lima['IDDIST'].unique()))

#### 2.3.2. Dataset de trabajo

In [ ]:
escala = 100*(geoData['precio_soles']/geoData['precio_soles'].mean())
scheme = mc.Quantiles(geoData['precio_soles'], k=5)

ax = gplt.polyplot(
    shp_lima,
    zorder=-1,
    linewidth=1,
    edgecolor='white',
    facecolor='lightgray',
    figsize=(12, 12)
)
gplt.pointplot(
    geoData, 
    ax=ax, 
    hue='precio_soles', 
    scale=escala, 
    scheme=scheme, 
    limits=(1, 15),
    cmap='inferno_r'

)

plt.show()

## 3. EDA

### 3.1. Parte 1: corrección de datos

+ *Limpieza y transformaciones previas*

In [ ]:
df_transf = df_casas.copy()
df_transf.head(3)

#### 3.1.1. Missing data

##### a. Check con Pandas

+ **Columnas con valores missing**

In [ ]:
for x in df_transf.columns[df_transf.isnull().any()]:
    print(x)

+ **Número de valores missing por columna**

In [ ]:
df_transf.isna().sum()

+ **Validar campos "object"**

In [ ]:
df_transf.info()

In [ ]:
empty_string = ''
print(empty_string is None)

In [ ]:
# > ajuste para un campo "object"
df_transf['PROVINCIA'] = df_transf['PROVINCIA'].fillna('NE')
df_transf['PROVINCIA'] = df_transf['PROVINCIA'].replace('', 'NE')

df_transf['PROVINCIA'].isna().sum()

In [ ]:
# > para un campos "object"
obj_col = ['PROVINCIA', 'DISTRITO', 'estado_de_inmueble']
for c in obj_col:
    df_transf[c] = df_transf[c].fillna('NE')
    df_transf[c] = df_transf[c].replace('', 'NE')

df_transf.isna().sum()

##### b. Missingno

+ *Visualizar missings*

In [ ]:
df_test = pd.read_csv('./data/test_missingno.csv', delimiter=';')
missingno.matrix(df_test)

In [ ]:
missingno.matrix(df_test.sort_values(by='consent'))
# del df_test

#### 3.1.2. Limpieza de columnas

##### a. Inconsistencias campo-tipo de dato

+ *Validar match entre naturaleza del campo y tipo de dato*

In [ ]:
df_transf.info()

In [ ]:
# > detail
df_transf['nro_dormitorios'].unique()

In [ ]:
# > ajuste en tipo de dato
df_transf['nro_dormitorios'] = df_transf['nro_dormitorios'].str.extract(r'(\d+)')
df_transf['nro_dormitorios'] = df_transf['nro_dormitorios'].astype('int')

In [ ]:
# > campo ajustado
df_transf['nro_dormitorios'].unique()

##### b. Inconsistencias según campo (variables cuantitativas)

+ *Validar match entre naturaleza del campo y valor*

In [ ]:
df_transf.describe()

#### 3.1.3. Transformaciones previas

+ *Transformaciones para antes de análisis*

##### a. Dummies sobre variables cualitativas

In [ ]:
# > columnas cualitativas
qual_cols = obj_col[:]
print(qual_cols)

In [ ]:
# > save columnas cualitativas
for c in qual_cols:
    print(c, '-->', len(df_transf[c].unique()))
    df_transf[c + '_'] = df_transf[c]

df_transf.info()

In [ ]:
# > get dummies
df_transf = pd.get_dummies(df_transf, columns=qual_cols, drop_first=False)

In [ ]:
# > after get dummies
print(df_transf.shape)
df_transf.head(3)

#### 3.1.4. Saving cleaned dataset

+ *Guardado de datos para análisis en segunda parte de EDA*

In [ ]:
df_transf.to_csv('./temp/data_cleaned.csv', index=False)
# del df_casas; del df_transf

### 3.2. Parte 2: EDA

In [ ]:
# > import cleaned data
df_clean = pd.read_csv('./temp/data_cleaned.csv').set_index('IDDIST')

print(df_clean.shape)
df_clean.head(3)

In [ ]:
# > cleaned dataframe shape
df_clean.shape

In [ ]:
# > null values
df_clean.isna().sum().sum()

#### 3.2.1. Variables cuantitativas

##### a. Estadísticos

In [ ]:
quan_cols = [
    'antiguedad',
    'nro_banios',
    'nro_pisos',
    'nro_cocheras',
    'nro_dormitorios',
    'precio_soles',
    'area_constr_m2',
    'area_total_m2'
]

df_clean[quan_cols].describe()

##### b. Distribuciones

In [ ]:
g = sns.PairGrid(
    df_clean[quan_cols].reset_index(drop=True),
    diag_sharey=False,
)
g.map_diag(sns.kdeplot)
g.map_upper(sns.scatterplot, markers='.', s=5)
g.map_lower(sns.regplot, scatter=False, line_kws={'linewidth':1})
g.add_legend()
plt.show()

##### c. Outliers

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(20, 10), sharex=True)

# 1) antiguedad
sns.boxplot(y=quan_cols[0], data=df_clean, ax=ax[0,0])
# 2) nro_banios
sns.boxplot(y=quan_cols[1], data=df_clean, ax=ax[0,1])
# 3) nro_pisos
sns.boxplot(y=quan_cols[2], data=df_clean, ax=ax[0,2])
# 4) nro_cocheras
sns.boxplot(y=quan_cols[3], data=df_clean, ax=ax[0,3])
# 5) nro_dormitorios
sns.boxplot(y=quan_cols[4], data=df_clean, ax=ax[1,0])
# 6) precio_soles
sns.boxplot(y=quan_cols[5], data=df_clean, ax=ax[1,1])
# 7) area_constr_m2
sns.boxplot(y=quan_cols[6], data=df_clean, ax=ax[1,2])
# 8) area_total_m2
sns.boxplot(y=quan_cols[7], data=df_clean, ax=ax[1,3])

plt.show()

#### 3.2.2. Variables cualitativas

In [ ]:
qual_cols = ['PROVINCIA_', 'DISTRITO_', 'estado_de_inmueble_']
df_clean[qual_cols].head(5)

##### a. Distribution variables cualitativas

In [ ]:
# PROVINCIA_
df_clean['PROVINCIA_'].value_counts(normalize=True)

In [ ]:
# > DISTRITO_ (solo representacion 1% a mas)
(y := df_clean['DISTRITO_'].value_counts(normalize=True))[y > 0.050]

In [ ]:
# estado_de_inmueble_
df_clean['estado_de_inmueble_'].value_counts(normalize=True)

+ **Visualmente**

In [ ]:
ytitle = 'Porcentaje'
fig, ax = plt.subplots(2, 2, figsize=(15, 10))

# 1) PROVINCIA_
percentPlot.seaPlot(df_clean, 'PROVINCIA_', ytitle, ax=ax[0,0])
# 2) nro_dormitorios_
percentPlot.seaPlot(df_clean, 'nro_dormitorios', ytitle, ax=ax[0,1])
# 3) estado_de_inmueble_
percentPlot.seaPlot(df_clean, 'estado_de_inmueble_', ytitle, ax=ax[1,0], palette=colors)
# 4) DISTRITO_
percentPlot.seaPlot(df_clean, 'DISTRITO_', ytitle, ax=ax[1,1], palette=colors)

ax[1,1].set_xticklabels([])
ax[1,0].set_xticklabels([])

plt.show()

##### b. Cross-tabulation

In [ ]:
df_clean['PROVINCIA_'].value_counts()

In [ ]:
# > count
pd.crosstab(df_clean['PROVINCIA_'], df_clean['estado_de_inmueble_'])

In [ ]:
# > relativo horizontalmente
pd.crosstab(df_clean['PROVINCIA_'], df_clean['estado_de_inmueble_'], normalize='index')

In [ ]:
# > relativo verticalmente
pd.crosstab(df_clean['PROVINCIA_'], df_clean['estado_de_inmueble_'], normalize='columns')

##### b. Cruce variables-cuantitativas

In [ ]:
# > un agrupamiento
df_clean.groupby(['PROVINCIA_'], axis=0).agg({'antiguedad': ['mean', 'count'], 'precio_soles': [np.max, np.min]})

In [ ]:
# > doble agrupamiento
df_clean.groupby(['PROVINCIA_', 'estado_de_inmueble_'], axis=0).agg({'antiguedad': ['mean', 'count'], 'precio_soles': [np.median, np.min]})

+ **Visualmente**

In [ ]:
sns.catplot(
    x='precio_soles',
    y='estado_de_inmueble_',
    kind='box',
    orient='h',
    height=18,
    palette=colors,
    data=df_clean
)
plt.show()

#### 3.2.3. Correlation

##### a. Correlación entre features

In [ ]:
# > values
cmatrixt = df_clean.drop('precio_soles', axis=1).corr().abs()
cmatrixt.head()

In [ ]:
# > correlation plot
plt.figure(figsize=(20, 10))
heatmap = sns.heatmap(cmatrixt.iloc[1:], vmin=0, vmax=1, cmap='viridis')
heatmap.set_title('Correlation between features', fontdict={'fontsize':18}, pad=16)
plt.show()

##### b. Correlación entre target y features

In [ ]:
# > values
cmatrix = df_clean.corr()[['precio_soles']].abs().sort_values(by='precio_soles', ascending=False)
cmatrix = cmatrix[cmatrix['precio_soles'] > 0.10]
cmatrix.head()

In [ ]:
# > correlation plot
plt.figure(figsize=(7, 10))
heatmap = sns.heatmap(cmatrix.iloc[1:], vmin=0, vmax=1, annot=True, cmap='viridis')
heatmap.set_title('Correlation between price and features', fontdict={'fontsize':18}, pad=16)
plt.show()

#### 3.2.4. Feature engineering

##### a. External coordenadas

In [ ]:
main_p = pd.read_csv('./data/main_points.csv', delimiter=';').set_index('IDDIST')

main_p['mp_m'] = main_p['mp_m'].apply(literal_eval)
main_p['mp_c'] = main_p['mp_c'].apply(literal_eval)

main_p[['mp_m_x', 'mp_m_y']] = pd.DataFrame(main_p['mp_m'].to_list(), index=main_p.index)
main_p[['mp_c_x', 'mp_c_y']] = pd.DataFrame(main_p['mp_c'].to_list(), index=main_p.index)

main_p.head()

##### b. Joins

+ *Presencia de campo en común*

In [ ]:
# > temporal dataframes
tmp_df_clean = df_clean.copy().reset_index()
tmp_main_p = main_p.copy().reset_index()

tmp_campos = ['IDDIST', 'latitud', 'longitud', 'precio_soles']

+ **Por campos específicas**

In [ ]:
tmp_df_clean[tmp_campos].merge(tmp_main_p, how='inner', left_on='IDDIST', right_on='IDDIST').head()

+ **Por campo en común**

In [ ]:
tmp_df_clean[tmp_campos].merge(tmp_main_p, how='inner', on='IDDIST').head()

+ **Por campo en común y sufijos**

In [ ]:
pd.merge(tmp_df_clean[tmp_campos], tmp_main_p, how='inner', on='IDDIST', suffixes=('_izq', '_der')).head()

In [ ]:
print(df_clean.index.name, '---', main_p.index.name)

In [ ]:
df_coor = df_clean[['latitud', 'longitud', 'precio_soles']].join(main_p)
df_coor.reset_index(inplace=True)

df_coor.head()

+ **Visualmente**

In [ ]:
# > geodataframes
geoData_clean = gpd.GeoDataFrame(
    df_coor.copy(),
    geometry=gpd.points_from_xy(df_coor['longitud'], df_coor['latitud'])
)
geoData_main_m = gpd.GeoDataFrame(
    df_coor.copy(),
    geometry=gpd.points_from_xy(df_coor['mp_m_y'], df_coor['mp_m_x'])
)
geoData_main_c = gpd.GeoDataFrame(
    df_coor.copy(),
    geometry=gpd.points_from_xy(df_coor['mp_c_y'], df_coor['mp_c_x'])
)

# > scale points
escala = 100*(geoData_clean['precio_soles']/geoData_clean['precio_soles'].mean())
scheme = mc.Quantiles(geoData_clean['precio_soles'], k=5)

# > geoplots
ax = gplt.polyplot(
    shp_lima,
    zorder=-1,
    linewidth=1,
    edgecolor='white',
    facecolor='lightgray',
    figsize=(20, 20)
)
gplt.pointplot(
    geoData_clean, 
    ax=ax,
    hue='precio_soles', 
    scale=escala, 
    scheme=scheme, 
    cmap='inferno_r',
)
gplt.pointplot(
    geoData_main_m, 
    ax=ax,
    s=7
)
gplt.pointplot(
    geoData_main_c, 
    ax=ax,
    s=7
)
plt.show()

##### c. Distancia entre coordenadas

+ **Geopy distance**

In [ ]:
np_lon_lat = df_coor[['latitud', 'longitud']].values.tolist()
np_mpm = df_coor['mp_m'].values.tolist()
np_mpc = df_coor['mp_c'].values.tolist()

In [ ]:
# => casa de ejemplo
print('distance casa-mun:', distance.distance(np_mpm[-1], np_lon_lat[-1]).km)
print('distance casa-com:', distance.distance(np_mpc[-1], np_lon_lat[-1]).km)

print('casa en venta....:', np_lon_lat[-1])
print('municipalidad....:', np_mpm[-1])
print('estacion.........:', np_mpc[-1])

+ **Campo distancia**

In [ ]:
df_coor['km_m'] = km_bpoints(df_coor, point_xy='mp_m', lon_column='longitud', lat_column='latitud')
df_coor['km_c'] = km_bpoints(df_coor, point_xy='mp_c', lon_column='longitud', lat_column='latitud')

In [ ]:
df_coor.head(5)

## 4. Model development inputs

### 4.1. Main points saving

In [ ]:
# > distritos match data casas
print(len(main_p.index))
print(len(df_clean.index.unique()))
main_p[['mp_m', 'mp_c']].head()

In [ ]:
# > to pickle
main_p[['mp_m', 'mp_c']].to_pickle('./temp/main_p.pkl')

### 4.2. Dataset for model development

+ *Dataset limpio con campos originales*

In [ ]:
df_clean = pd.read_csv('./temp/data_cleaned.csv')
df_clean.head()

In [ ]:
# > obtener campos originales de tabla raw
orig_col_length = len(pd.read_csv('./data/casas_data.csv', delimiter=';').columns)
print(orig_col_length)

In [ ]:
# > filtrar columnas
df_model = df_clean[df_clean.columns[:orig_col_length]]

print(df_model.shape)

In [ ]:
# > guardado de datos
df_model.to_csv('./temp/data_model.csv', index=False)
# del df_model; del df_clean; del df_coor